# CNN+LSTM + Dense 모델을 작성해 주가 예측
삼성전자 증권표준코드 005930 를 이용해서 주가 예측

In [ ]:
!pip install finance-datareader

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import FinanceDataReader as fdr
import warnings
warnings.filterwarnings('ignore')

STOCK_CODE='005930'
stock=fdr.DataReader(STOCK_CODE)

print(stock.head(3))
print(stock.tail(3))
print()

# Date열을 이용해 연도별 주가 변동 시각화
# Date열을 연,월,일로 분리해 새로운 열을 추가
print(stock.index)
stock['year']=stock.index.year
stock['month']=stock.index.month
stock['day']=stock.index.day
print(stock.head(3))
print(stock.shape)

plt.figure(figsize=(6,4))
sns.lineplot(y=stock['Close'],x=stock.year)
plt.xlabel('time')
plt.ylabel('price')
plt.show()

In [ ]:
time_step=[['2000','2005'],['2005','2010'],['2010','2015'],['2015','2023']]
fig,axes=plt.subplots(2,2)
fig.set_size_inches(10,6)

for i in range(4):
    ax=axes[i//2,i%2]
    df=stock.loc[(stock.index>time_step[i][0])&(stock.index<time_step[i][1])]
    sns.lineplot(y=df['Close'],x=df.index,ax=ax)
    plt.xlabel('time')
    plt.ylabel('price')
plt.tight_layout()
plt.show()

In [ ]:
# 전처리: Open High Low Close Volumne
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scale_cols = ['Open','High','Low','Close','Volume']
df_scaled = scaler.fit_transform(stock[scale_cols])
df= pd.DataFrame(df_scaled,columns=scale_cols)
print(df.head(2))
print()

# train_test split
from sklearn.model_selection import train_test_split
x_train,x_test, y_train,y_test = train_test_split(df.drop('Close',1),df['Close'],test_size=0.2,random_state=12,shuffle=False)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape) #(4800, 4) (1200, 4) (4800,) (1200,)
print()
print(x_train[:2])
print()
print(y_train[:2])



In [ ]:
# CNN+ - CONV1D를 사용해야 함
# Dataset을 활용해서 sequence dataset을 구성
import tensorflow as tf

def window_dataset_func(series,window_size,batch_size,shuffle):
    series = tf.expand_dims(series,axis=-1)
    ds=tf.data.Dataset.from_tensor_slices(series) #예) (60000,28, 28) -> 28*28의 크기 6만개로 슬라이싱
    ds=ds.window(window_size+1, shift=1, drop_remainder=True) #Dataset.window(그룹화할 윈도우 크기, 이동수)- shift 값에 따라 그리고, remainder가 true냐 false임에 따라 슬라이싱해서 시계열 데이터를 표현할 수 잇음
    ds= ds.flat_map(lambda w:w.batch(window_size+1)) # map- 함수를 실행하는 함수
    if shuffle:
        ds=ds.shuffle(buffer_size=1000)
    ds=ds.map(lambda w:(w[:-1],w[-1]))
    return ds.batch(batch_size).prefetch(1) # prefetch(1)-dataset 1개(20단위 데이터)를 미리 준비해 둠. 훈련속도를 더 빠르게 처리
    #ds. batch(batch_size) - 큰데이터는 메모리에 한번에 올릴수 없으므로 batch_size만큼 나누어 학습을 시켜야 한다.


WINDOW_SIZE=20
BATCH_SIZE=32
train_data=window_dataset_func(y_train,WINDOW_SIZE,BATCH_SIZE,True) #train data는 shffuling하기 -꼭 안섞을 필요는 없음 y_train은 (4800,)짜리 label
test_data=window_dataset_func(y_test,WINDOW_SIZE,BATCH_SIZE,False) #그러나 test는 원래의 데이터를 그대로 유지해야 한다- 검증데이터이기 때문
print(train_data)
print(test_data)

for data in train_data.take(1):
    print(data[0].shape) #(32, 20, 1) - batch_size,window_size,feature
    print(data[1].shape) #(32, 1) - batch_size,feature
    #이런식으로 끊어가면서 학습하는 것- DataSet이 이것을 도와주는 것임


In [ ]:
# model
from keras.models import Sequential
from keras.layers import Dense, LSTM,Conv1D
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.losses import Huber
from keras.optimizers import Adam #전통적인 방식에서는 최소제곱법을 사용, optimizers를 import

model= Sequential([
    Conv1D(filters=32,kernel_size=5,padding='causal', activation='relu', input_shape=[WINDOW_SIZE,1]),  # 시간 순서를 위반하지 않는 시계열 데이터를 처리시 효과적
    LSTM(16,activation='tanh'),
    Dense(16,activation='relu'),
    Dense(1)
])


print(model.summary())

loss=Huber()
optimizer=Adam(learning_rate=0.0005)
model.compile(loss=loss,optimizer=optimizer,metrics=['mse'])

es=EarlyStopping(monitor='val_loss', patience=30)

history=model.fit(train_data,epochs=100, batch_size=32, validation_data=(test_data),verbose=2,callbacks=[es])


In [ ]:
pred= model.predict(test_data)
print('예측값 : ', pred[:10].flatten())
print('실제값 : ', np.asarray(y_test)[:10])

plt.figure(figsize=(8,5))
plt.plot(np.asarray(y_test)[20:], label='real')
plt.plot(pred,label='pred',c='blue')
plt.legend()
plt.show()